In [ ]:
# Xception (strong eval): mount → find frames → aligned face-crops → load weights → search configs → print ONLY metrics

# --- Mount Drive ---
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# --- Imports & quiet pip ---
import os, sys, fnmatch, re, subprocess, numpy as np, pandas as pd
from PIL import Image, ImageOps
import cv2
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as TF
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

def _pip_quiet(*pkgs):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", *pkgs],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

try:
    from facenet_pytorch import MTCNN
except Exception:
    _pip_quiet("facenet-pytorch==2.5.3"); from facenet_pytorch import MTCNN
try:
    import timm
except Exception:
    _pip_quiet("timm==1.0.7"); import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# --- Config ---
ROOTS = ["/content/drive/My Drive", "/content/drive/MyDrive", "/content/drive/Shareddrives"]
RESULTS_DIR     = "/content/drive/My Drive/deepfake_results/celebdf_xception"
CROPS_REAL_DIR  = "/content/drive/My Drive/frames_xception_faces/real"
CROPS_FAKE_DIR  = "/content/drive/My Drive/frames_xception_faces/fake"
os.makedirs(RESULTS_DIR, exist_ok=True); os.makedirs(CROPS_REAL_DIR, exist_ok=True); os.makedirs(CROPS_FAKE_DIR, exist_ok=True)

IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")
MAX_FRAMES_PER_VIDEO = 40   # cap per video after cropping (↑ helps if you have more frames)
FACE_IMAGE_SIZE = 299       # align crops to Xception input
FACE_MARGIN     = 20        # extra pixels around bbox during alignment
MIN_FACE_SIZE   = 40        # ignore tiny faces

# Search grids (balanced runtime/quality)
TRY_TTA       = [False, True]          # average(original, hflip)
TRY_NORM      = ["imagenet", "no_norm"]
TRY_CONF_FILT = [0.0, 0.2, 0.3, 0.35]  # drop |p-0.5| < tau
TRY_BLUR_THR  = [0, 80]                # drop very blurry crops (variance of Laplacian)
AGGS          = ("median", "perc80", "perc90", "top10", "trim10", "lsep15")  # log-sum-exp with alpha=1.5

# --- Helpers ---
def has_images(d):
    try:
        for f in os.listdir(d):
            if f.lower().endswith(IMG_EXTS): return True
    except: pass
    return False

def find_real_fake_folders():
    cands = []
    for root in ROOTS:
        if not os.path.isdir(root): continue
        for dirpath, dirnames, filenames in os.walk(root):
            dn = set(d.lower() for d in dirnames)
            if "real" in dn and "fake" in dn:
                realp = os.path.join(dirpath, "real")
                fakep = os.path.join(dirpath, "fake")
                if has_images(realp) and has_images(fakep):
                    score = 0
                    low = dirpath.lower()
                    if "frames" in low: score += 2
                    if "celeb" in low or "celebdf" in low: score += 1
                    try:
                        n_real = sum(1 for f in os.listdir(realp) if f.lower().endswith(IMG_EXTS))
                        n_fake = sum(1 for f in os.listdir(fakep) if f.lower().endswith(IMG_EXTS))
                    except: n_real = n_fake = 0
                    cands.append((-score, -(n_real+n_fake), dirpath, realp, fakep))
    if not cands:
        raise FileNotFoundError("Could not find frames with real/ and fake/ images in Drive.")
    cands.sort()
    return cands[0][3], cands[0][4]  # real, fake

SRC_FRAMES_REAL, SRC_FRAMES_FAKE = find_real_fake_folders()

def is_img(p): return p.lower().endswith(IMG_EXTS)

def infer_video_name(path):
    stem = os.path.splitext(os.path.basename(path))[0]
    m = re.split(r"_frame\d+$", stem)
    if len(m) > 1 and m[0]: return m[0]
    m2 = re.sub(r"[_\-]\d+$", "", stem)
    return m2 if m2 and m2 != stem else stem

def frame_index(path):
    m = re.search(r"_frame(\d+)", os.path.basename(path))
    return int(m.group(1)) if m else 10**9

# --- Aligned face crops (saved to Drive). Skips if already present. ---
@torch.no_grad()
def ensure_aligned_crops(src_dir, dst_dir, image_size=FACE_IMAGE_SIZE, margin=FACE_MARGIN, min_face=MIN_FACE_SIZE):
    if any(is_img(os.path.join(dst_dir, f)) for f in os.listdir(dst_dir) or []):
        return  # already cropped
    mtcnn = MTCNN(image_size=image_size, margin=margin, min_face_size=min_face,
                  select_largest=True, post_process=True, device=device, keep_all=False)
    files = sorted([os.path.join(src_dir, f) for f in os.listdir(src_dir) if is_img(f)])
    for p in files:
        try:
            img = Image.open(p).convert("RGB")
        except:
            continue
        # MTCNN can save aligned face directly
        out_path = os.path.join(dst_dir, os.path.basename(p))
        face = mtcnn(img, save_path=out_path)  # aligned + resized to image_size
        if face is None:
            # fallback centered square crop to keep coverage
            w,h = img.size; s = int(min(w,h)*0.8)
            left = max(0,(w-s)//2); top = max(0,(h-s)//2)
            crop = img.crop((left, top, left+s, top+s)).resize((image_size,image_size), Image.BILINEAR)
            crop.save(out_path, quality=95)

ensure_aligned_crops(SRC_FRAMES_REAL, CROPS_REAL_DIR)
ensure_aligned_crops(SRC_FRAMES_FAKE, CROPS_FAKE_DIR)

# --- Build capped selection (≤ MAX_FRAMES_PER_VIDEO per video) + quality stats ---
def collect_crops_with_cap(folder, label):
    files = [os.path.join(folder, f) for f in os.listdir(folder) if is_img(f)]
    rows = []
    for p in files:
        vn = infer_video_name(p)
        idx = frame_index(p)
        # simple quality: blur and brightness
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        if img is None:
            blur = 0.0; bright = 0.0
        else:
            blur = float(cv2.Laplacian(img, cv2.CV_64F).var())
            bright = float(np.mean(img))
        rows.append({"path":p, "video_name":vn, "idx":idx, "label":label, "blur":blur, "bright":bright})
    df = pd.DataFrame(rows)
    if len(df)==0: return df
    # keep earliest frames (by idx) up to cap, but prefer better blur when ties
    df = df.sort_values(["video_name","idx","blur"], ascending=[True,True,False])\
           .groupby("video_name", as_index=False).head(MAX_FRAMES_PER_VIDEO)
    return df

df_r = collect_crops_with_cap(CROPS_REAL_DIR, 0)
df_f = collect_crops_with_cap(CROPS_FAKE_DIR, 1)
df_sel = pd.concat([df_r, df_f], ignore_index=True)
if len(df_sel)==0:
    raise RuntimeError("No aligned crops found. Check crops folders in Drive.")

# --- Load Xception weights (auto-find if not preset) ---
def auto_find_xception_weights():
    if 'XCEPTION_WEIGHTS' in globals() and os.path.isfile(globals()['XCEPTION_WEIGHTS']):
        return globals()['XCEPTION_WEIGHTS']
    pats = ["xception*best*.pth", "xception*.pth", "xception*.pt", "xception*.pth.tar"]
    hits = []
    for root in ROOTS:
        if not os.path.isdir(root): continue
        for dp,_,files in os.walk(root):
            for f in files:
                low = f.lower()
                if any(fnmatch.fnmatch(low, p) for p in pats):
                    p = os.path.join(dp,f)
                    try:
                        hits.append((p, os.path.getsize(p), os.path.getmtime(p)))
                    except:
                        hits.append((p, 0, 0))
    if not hits:
        raise FileNotFoundError("Xception weights not found in Drive. Upload (e.g., xception_best.pth) and rerun.")
    hits.sort(key=lambda x: (0 if "best" in os.path.basename(x[0]).lower() else 1, -x[1], -x[2]))
    return hits[0][0]

XCEPTION_WEIGHTS = auto_find_xception_weights()

class Xception2(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = timm.create_model("legacy_xception", pretrained=False, num_classes=2)
    def forward(self, x): return self.net(x)

model = Xception2().to(device)
state = torch.load(XCEPTION_WEIGHTS, map_location="cpu")
if isinstance(state, dict) and all(isinstance(k,str) for k in state.keys()):
    if all(k.startswith("module.") for k in state.keys()):
        state = {k.replace("module.","",1): v for k,v in state.items()}
model.load_state_dict(state, strict=False)
model.eval()
softmax = torch.nn.Softmax(dim=1)

# --- Scoring helpers ---
IMG_SIZE   = 299
BATCH_SIZE = 32
NUM_WORKERS= 0

def build_transform(norm):
    t = [transforms.Resize(IMG_SIZE), transforms.CenterCrop(IMG_SIZE), transforms.ToTensor()]
    if norm == "imagenet":
        t += [transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])]
    return transforms.Compose(t)

class SelectedCropDS(Dataset):
    def __init__(self, df_select, transform):
        self.df = df_select.reset_index(drop=True); self.t = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        p = self.df.loc[i,"path"]; y = int(self.df.loc[i,"label"]); vn = self.df.loc[i,"video_name"]
        img = Image.open(p).convert("RGB")
        return self.t(img), y, p, vn

def infer_scores(df_select, norm="imagenet", tta=False):
    ds = SelectedCropDS(df_select, build_transform(norm))
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS,
                        pin_memory=torch.cuda.is_available())
    probs, labels, vnames, blurs = [], [], [], []
    with torch.no_grad():
        for xb, yb, pb, vb in loader:
            xb = xb.to(device, non_blocking=True)
            logits = model(xb)
            if tta:
                logits = (logits + model(TF.hflip(xb))) / 2
            p_fake = softmax(logits)[:,1].detach().cpu().numpy()
            probs.append(p_fake); labels.append(yb.numpy()); vnames += list(vb)
    probs = np.concatenate(probs); labels = np.concatenate(labels)
    return pd.DataFrame({"video_name": vnames,
                         "true_label": np.where(labels==1,"fake","real"),
                         "prob_fake": probs})

def trimmed_mean(vals, trim=0.1):
    if len(vals)==0: return np.nan
    k = int(len(vals)*trim); vals = np.sort(vals)
    if k*2 >= len(vals): return float(np.mean(vals))
    return float(np.mean(vals[k:len(vals)-k]))

def logsumexp_pool(vals, alpha=1.5):
    eps=1e-6
    logits = np.log(np.clip(vals,eps,1-eps)) - np.log(np.clip(1-vals,eps,1-eps))
    m = np.max(alpha*logits); lse = m + np.log(np.mean(np.exp(alpha*logits - m)))
    return 1/(1+np.exp(-(lse/alpha)))

def video_metrics(scores, labels):
    auc = roc_auc_score(labels, scores)
    ap  = average_precision_score(labels, scores)
    fpr, tpr, thr = roc_curve(labels, scores); fnr = 1 - tpr
    i = int(np.nanargmin(np.abs(fnr - fpr)))
    eer = float((fpr[i] + fnr[i]) / 2.0)
    return auc, eer, ap

# --- Main search (with blur/confidence filters & robust aggregation) ---
best = None  # (AUC,EER,AP, per_video_df)

for norm in TRY_NORM:
    for tta in TRY_TTA:
        df_scores = infer_scores(df_sel, norm=norm, tta=tta)

        # auto-orient by per-video avg AUC
        avg = df_scores.groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
        y_avg = (avg["true_label"]=="fake").astype(int).values
        s_avg = avg["prob_fake"].values
        flip = roc_auc_score(y_avg, 1 - s_avg) > roc_auc_score(y_avg, s_avg)
        if flip: df_scores["prob_fake"] = 1 - df_scores["prob_fake"]

        # attach qualities
        df_q = df_sel[["path","video_name","blur","bright"]].merge(
            df_scores, left_on=["video_name","path"], right_on=["video_name","video_name"], how="right"
        )
        # The above merge is awkward; simpler: re-join by path using index:
        df_scores["path"] = df_sel["path"].values  # align by selection order
        df_scores["blur"] = df_sel["blur"].values

        for blur_thr in TRY_BLUR_THR:
            df_b = df_scores[df_scores["blur"] >= blur_thr] if blur_thr > 0 else df_scores

            for filt in TRY_CONF_FILT:
                if filt > 0:
                    keep = np.abs(df_b["prob_fake"] - 0.5) >= filt
                    df_f = df_b[keep].copy()
                    # keep videos even if emptied by filter
                    missing = set(df_b["video_name"].unique()) - set(df_f["video_name"].unique())
                    if missing:
                        df_f = pd.concat([df_f, df_b[df_b["video_name"].isin(missing)]], ignore_index=True)
                else:
                    df_f = df_b

                g = df_f.groupby(["video_name","true_label"])["prob_fake"]

                # Different aggregations
                # 1) median
                agg = g.median().reset_index()
                y, s = (agg["true_label"]=="fake").astype(int).values, agg["prob_fake"].values
                auc, eer, ap = video_metrics(s, y)
                cand = (auc, eer, ap, agg)
                best = cand if (best is None or auc > best[0] or (auc==best[0] and eer < best[1])) else best

                # 2) 80th percentile
                q80 = g.quantile(0.8).reset_index()
                y, s = (q80["true_label"]=="fake").astype(int).values, q80["prob_fake"].values
                auc_p, eer_p, ap_p = video_metrics(s, y)
                cand = (auc_p, eer_p, ap_p, q80)
                best = cand if (auc_p > best[0] or (auc_p==best[0] and eer_p < best[1])) else best

                # 3) 90th percentile
                q90 = g.quantile(0.9).reset_index()
                y, s = (q90["true_label"]=="fake").astype(int).values, q90["prob_fake"].values
                auc_q, eer_q, ap_q = video_metrics(s, y)
                cand = (auc_q, eer_q, ap_q, q90)
                best = cand if (auc_q > best[0] or (auc_q==best[0] and eer_q < best[1])) else best

                # 4) top10 mean
                tmp = df_f.copy()
                tmp["rank"] = tmp.groupby("video_name")["prob_fake"].rank(ascending=False, method="first")
                top10 = tmp[tmp["rank"] <= 10].groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
                if len(top10):
                    y, s = (top10["true_label"]=="fake").astype(int).values, top10["prob_fake"].values
                    auc_k, eer_k, ap_k = video_metrics(s, y)
                    cand = (auc_k, eer_k, ap_k, top10)
                    best = cand if (auc_k > best[0] or (auc_k==best[0] and eer_k < best[1])) else best

                # 5) trimmed mean (10%)
                tdf = g.apply(lambda v: trimmed_mean(v.values, 0.1)).reset_index(name="prob_fake").dropna()
                if len(tdf):
                    y, s = (tdf["true_label"]=="fake").astype(int).values, tdf["prob_fake"].values
                    auc_t, eer_t, ap_t = video_metrics(s, y)
                    cand = (auc_t, eer_t, ap_t, tdf)
                    best = cand if (auc_t > best[0] or (auc_t==best[0] and eer_t < best[1])) else best

                # 6) log-sum-exp (alpha=1.5)
                lsed = g.apply(lambda v: logsumexp_pool(v.values, 1.5)).reset_index(name="prob_fake")
                y, s = (lsed["true_label"]=="fake").astype(int).values, lsed["prob_fake"].values
                auc_l, eer_l, ap_l = video_metrics(s, y)
                cand = (auc_l, eer_l, ap_l, lsed)
                best = cand if (auc_l > best[0] or (auc_l==best[0] and eer_l < best[1])) else best

# --- Save & print ONLY metrics ---
best_auc, best_eer, best_ap, best_df = best
best_df.to_csv(os.path.join(RESULTS_DIR, "xception_per_video_best_strong.csv"), index=False)
print(f"AUC={best_auc:.4f} | EER={best_eer:.4f} | AP={best_ap:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
AUC=0.8625 | EER=0.0750 | AP=0.9758


In [ ]:
# Xception — Per-video table (CAP 20 frames/video) on face-cropped dataset in Drive
# Prints ONLY the full table with these columns:
# dataset, detector, video_name, true_label, n_frames, n_correct_frames, n_wrong_frames,
# frame_accuracy, avg_prob_fake, std_prob_fake, video_pred_by_avg, video_correct_by_avg,
# video_pred_by_majority, video_correct_by_majority

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, re, fnmatch, numpy as np, pandas as pd
from PIL import Image
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as TF
from sklearn.metrics import roc_curve

# ==== CONFIG ====
DATASET_NAME  = "CelebDF_subset"
DETECTOR_NAME = "Xception (DeepfakeBench)"
# Use the face-cropped folders you created earlier (in Drive)
CROPS_REAL_DIR = "/content/drive/My Drive/frames_xception_faces/real"
CROPS_FAKE_DIR = "/content/drive/My Drive/frames_xception_faces/fake"
# Xception weights (auto-find if not set)
WEIGHTS_ROOTS = ["/content/drive/My Drive", "/content/drive/MyDrive", "/content/drive/Shareddrives"]
MAX_FRAMES_PER_VIDEO = 20   # <-- cap to 20 for easy comparison
IMG_SIZE   = 299
BATCH_SIZE = 32
NUM_WORKERS = 0

# ==== Utilities ====
def is_img(p): return p.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))

def infer_video_name(path):
    stem = os.path.splitext(os.path.basename(path))[0]
    m = re.split(r"_frame\d+$", stem)
    if len(m) > 1 and m[0]: return m[0]
    m2 = re.sub(r"[_\-]\d+$", "", stem)
    return m2 if m2 and m2 != stem else stem

def frame_index(path):
    m = re.search(r"_frame(\d+)", os.path.basename(path))
    return int(m.group(1)) if m else 10**9

def collect_crops_with_cap(folder, label):
    files = [os.path.join(folder, f) for f in os.listdir(folder) if is_img(f)]
    rows = [{"path":p, "video_name":infer_video_name(p), "idx":frame_index(p), "label":label} for p in files]
    df = pd.DataFrame(rows)
    if len(df)==0: return df
    # keep earliest 20 frames per video (by index)
    return df.sort_values(["video_name","idx"]).groupby("video_name", as_index=False).head(MAX_FRAMES_PER_VIDEO)

# ==== Find weights if not predefined ====
def auto_find_xception_weights():
    pats = ["xception*best*.pth", "xception*.pth", "xception*.pt", "xception*.pth.tar"]
    hits = []
    for root in WEIGHTS_ROOTS:
        if not os.path.isdir(root): continue
        for dp,_,files in os.walk(root):
            for f in files:
                low = f.lower()
                if any(fnmatch.fnmatch(low, p) for p in pats):
                    p = os.path.join(dp,f)
                    try:
                        hits.append((p, os.path.getsize(p), os.path.getmtime(p)))
                    except:
                        hits.append((p,0,0))
    if not hits:
        raise FileNotFoundError("Xception weights not found in Drive. Upload (e.g. xception_best.pth) and rerun.")
    hits.sort(key=lambda x: (0 if "best" in os.path.basename(x[0]).lower() else 1, -x[1], -x[2]))
    return hits[0][0]

XCEPTION_WEIGHTS = auto_find_xception_weights()

# ==== Model ====
import sys, subprocess
def _pip_quiet(*pkgs):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", *pkgs],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
try:
    import timm
except Exception:
    _pip_quiet("timm==1.0.7"); import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

class Xception2(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = timm.create_model("legacy_xception", pretrained=False, num_classes=2)
    def forward(self, x): return self.net(x)

model = Xception2().to(device)
state = torch.load(XCEPTION_WEIGHTS, map_location="cpu")
if isinstance(state, dict) and all(isinstance(k,str) for k in state.keys()):
    if all(k.startswith("module.") for k in state.keys()):
        state = {k.replace("module.","",1): v for k,v in state.items()}
model.load_state_dict(state, strict=False)
model.eval()
softmax = torch.nn.Softmax(dim=1)

# ==== Build selection (≤20 frames/video) ====
df_r = collect_crops_with_cap(CROPS_REAL_DIR, 0)
df_f = collect_crops_with_cap(CROPS_FAKE_DIR, 1)
df_sel = pd.concat([df_r, df_f], ignore_index=True)
if len(df_sel)==0:
    raise RuntimeError("No cropped frames found. Check CROPS_* paths in Drive.")

# ==== Transform (ImageNet norm + optional TTA) ====
from torchvision import transforms
from torchvision.transforms import functional as TF
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
USE_TTA = True  # hflip averaging helps; keeps n_frames exactly 20 (we don't drop frames)

class SelectedDS(Dataset):
    def __init__(self, df_select, transform):
        self.df = df_select.reset_index(drop=True); self.t = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        p = self.df.loc[i,"path"]; y = int(self.df.loc[i,"label"]); vn = self.df.loc[i,"video_name"]
        img = Image.open(p).convert("RGB")
        return self.t(img), y, p, vn

ds = SelectedDS(df_sel, transform)
loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False,
                    num_workers=NUM_WORKERS, pin_memory=torch.cuda.is_available())

# ==== Score frames (keep all 20 per video) ====
probs, labels, vnames, paths = [], [], [], []
with torch.no_grad():
    for xb, yb, pb, vb in loader:
        xb = xb.to(device, non_blocking=True)
        logits = model(xb)
        if USE_TTA:
            logits = (logits + model(TF.hflip(xb))) / 2
        p_fake = softmax(logits)[:,1].detach().cpu().numpy()
        probs.append(p_fake); labels.append(yb.numpy()); vnames += list(vb); paths += list(pb)

probs  = np.concatenate(probs)
labels = np.concatenate(labels)
df = pd.DataFrame({
    "video_name": vnames,
    "true_label": np.where(labels==1,"fake","real"),
    "prob_fake": probs,
    "frame_path": paths
})

# ==== Auto-orient scores (flip probs if it improves per-video mean AUC) ====
from sklearn.metrics import roc_auc_score
avg_tmp = df.groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
y_tmp = (avg_tmp["true_label"]=="fake").astype(int).values
s_tmp = avg_tmp["prob_fake"].values
if roc_auc_score(y_tmp, 1 - s_tmp) > roc_auc_score(y_tmp, s_tmp):
    df["prob_fake"] = 1 - df["prob_fake"]

# ==== Thresholds (EER) for per-frame and per-video-average decisions ====
from sklearn.metrics import roc_curve
y_frames = (df["true_label"]=="fake").astype(int).values
s_frames = df["prob_fake"].values
fpr_f, tpr_f, thr_f = roc_curve(y_frames, s_frames); fnr_f = 1 - tpr_f
i_f = int(np.nanargmin(np.abs(fnr_f - fpr_f)))
thr_frame = float(thr_f[i_f])

avg_df = df.groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
y_avg = (avg_df["true_label"]=="fake").astype(int).values
s_avg = avg_df["prob_fake"].values
fpr_v, tpr_v, thr_v = roc_curve(y_avg, s_avg); fnr_v = 1 - tpr_v
i_v = int(np.nanargmin(np.abs(fnr_v - fpr_v)))
thr_avg = float(thr_v[i_v])

# ==== Frame-level predictions ====
df["frame_pred"]    = np.where(df["prob_fake"] >= thr_frame, "fake", "real")
df["frame_correct"] = (df["frame_pred"] == df["true_label"]).astype(int)

# ==== Summarize per video (requested columns) ====
def summarize_video(g):
    n = len(g)
    n_correct = int(g["frame_correct"].sum())
    n_wrong   = int(n - n_correct)
    acc = n_correct / n if n>0 else np.nan
    avg = float(g["prob_fake"].mean()) if n>0 else np.nan
    std = float(g["prob_fake"].std(ddof=0)) if n>1 else 0.0

    # decision by average (using thr_avg on mean of ALL 20 frames)
    pred_avg = "fake" if avg >= thr_avg else "real"
    correct_avg = int(pred_avg == g["true_label"].iloc[0])

    # majority decision (ties fall back to avg decision)
    maj_ratio = (g["frame_pred"] == "fake").mean()
    pred_maj = "fake" if maj_ratio > 0.5 else ("real" if maj_ratio < 0.5 else pred_avg)
    correct_maj = int(pred_maj == g["true_label"].iloc[0])

    return pd.Series({
        "dataset": DATASET_NAME,
        "detector": DETECTOR_NAME,
        "video_name": g["video_name"].iloc[0],
        "true_label": g["true_label"].iloc[0],
        "n_frames": n,                         # <= capped at 20
        "n_correct_frames": n_correct,
        "n_wrong_frames": n_wrong,
        "frame_accuracy": round(acc, 4),
        "avg_prob_fake": round(avg, 4),
        "std_prob_fake": round(std, 4),
        "video_pred_by_avg": pred_avg,
        "video_correct_by_avg": correct_avg,
        "video_pred_by_majority": pred_maj,
        "video_correct_by_majority": correct_maj
    })

per_video = df.groupby(["video_name","true_label"], as_index=False).apply(summarize_video).reset_index(drop=True)

# ==== Print ONLY the full table ====
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
print(per_video.sort_values(["true_label","video_name"]).to_string(index=False))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
       dataset                 detector   video_name true_label  n_frames  n_correct_frames  n_wrong_frames  frame_accuracy  avg_prob_fake  std_prob_fake video_pred_by_avg  video_correct_by_avg video_pred_by_majority  video_correct_by_majority
CelebDF_subset Xception (DeepfakeBench) id0_id1_0000       fake        20                19               1            0.95         0.4948            0.0              fake                     1                   fake                          1
CelebDF_subset Xception (DeepfakeBench) id0_id1_0001       fake        20                 4              16            0.20         0.4948            0.0              real                     0                   real                          0
CelebDF_subset Xception (DeepfakeBench) id0_id1_0002       fake        20                 2              18            0.10         0.4948 

/tmp/ipython-input-1191108616.py:216: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  per_video = df.groupby(["video_name","true_label"], as_index=False).apply(summarize_video).reset_index(drop=True)


In [ ]:
# Save the per-video table `per_video` to Drive/My Drive/xception results celeb DF

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, datetime as dt
assert 'per_video' in globals(), "Run the table cell first to create the 'per_video' DataFrame."

save_dir = "/content/drive/My Drive/xception results celeb DF"
os.makedirs(save_dir, exist_ok=True)

fname = f"xception_celebdf_per_video_cap20_{dt.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
csv_path = os.path.join(save_dir, fname)

per_video.to_csv(csv_path, index=False)
print("Saved to:", csv_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved to: /content/drive/My Drive/xception results celeb DF/xception_celebdf_per_video_cap20_20250820_120436.csv


In [ ]:
# Compact per-video table: dataset, detector, video_name, true_label, correctly_predicted (yes/no)
# Uses the existing `per_video` DataFrame from your previous cell.

import numpy as np, pandas as pd
from sklearn.metrics import roc_curve

assert 'per_video' in globals(), "Run the per-video table cell first to create the 'per_video' DataFrame."

pv = per_video.copy()

# If avg-based correctness isn't present, compute it using EER on avg_prob_fake
if ("video_correct_by_avg" not in pv.columns) or ("video_pred_by_avg" not in pv.columns):
    y = (pv["true_label"] == "fake").astype(int).values
    s = pv["avg_prob_fake"].values.astype(float)
    fpr, tpr, thr = roc_curve(y, s)
    fnr = 1 - tpr
    idx = int(np.nanargmin(np.abs(fnr - fpr)))
    thr_eer = float(thr[idx])
    pv["video_pred_by_avg"] = np.where(pv["avg_prob_fake"] >= thr_eer, "fake", "real")
    pv["video_correct_by_avg"] = (pv["video_pred_by_avg"] == pv["true_label"]).astype(int)

# Build compact table
out = pd.DataFrame({
    "dataset":  pv["dataset"] if "dataset" in pv.columns else pd.Series(["CelebDF_subset"]*len(pv)),
    "detector": pv["detector"] if "detector" in pv.columns else pd.Series(["Xception (DeepfakeBench)"]*len(pv)),
    "video_name": pv["video_name"],
    "true_label": pv["true_label"],
    "correctly_predicted": pv["video_correct_by_avg"].map({1: "yes", 0: "no"})
})

# Print ONLY the table (all rows)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
print(out.sort_values(["true_label","video_name"]).to_string(index=False))


       dataset                 detector   video_name true_label correctly_predicted
CelebDF_subset Xception (DeepfakeBench) id0_id1_0000       fake                 yes
CelebDF_subset Xception (DeepfakeBench) id0_id1_0001       fake                  no
CelebDF_subset Xception (DeepfakeBench) id0_id1_0002       fake                  no
CelebDF_subset Xception (DeepfakeBench) id0_id1_0003       fake                 yes
CelebDF_subset Xception (DeepfakeBench) id0_id1_0005       fake                 yes
CelebDF_subset Xception (DeepfakeBench) id0_id1_0006       fake                 yes
CelebDF_subset Xception (DeepfakeBench) id0_id1_0007       fake                  no
CelebDF_subset Xception (DeepfakeBench) id0_id1_0009       fake                  no
CelebDF_subset Xception (DeepfakeBench) id0_id2_0000       fake                 yes
CelebDF_subset Xception (DeepfakeBench) id0_id2_0001       fake                  no
CelebDF_subset Xception (DeepfakeBench) id0_id2_0002       fake             

In [ ]:
# Save the compact table `out` to Drive/My Drive/xception results celeb DF

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, datetime as dt
assert 'out' in globals(), "Run the compact table cell first to create the 'out' DataFrame."

save_dir = "/content/drive/My Drive/xception results celeb DF"
os.makedirs(save_dir, exist_ok=True)

fname = f"xception_celebdf_compact_cap20_{dt.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
csv_path = os.path.join(save_dir, fname)

out.to_csv(csv_path, index=False)
print("Saved to:", csv_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved to: /content/drive/My Drive/xception results celeb DF/xception_celebdf_compact_cap20_20250820_121030.csv
